## T Test

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import re
from collections import OrderedDict
import matplotlib.pyplot as plt


import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

import numpy as np
import pandas as pd
import scipy

In [2]:
data = pd.read_excel("data.xlsx", usecols = [0,1,2,3,4])
data = data.dropna()
data['heard_pitch'] = data['heard_pitch'].str.replace('[0-9 ]+', "")
# Convert the pitch heights into numbers to compute the difference 

pitch_dict = {
      'c':0, 'c#':1, 'db':1, 'd':2, 'd#':3, 'eb':3, 'e':4, 'fb':4, 'f':5, 'e#':5, 'f#':6, 'gb':6, 'g':5, 'g#':4, 'ab':4, 'a':3, 'a#':2, 'bb':2, 'b':1, 'cb':1, 'b#':0}

def pitch_to_int(x):
    if x in pitch_dict.keys():
        return pitch_dict[x]
    else:
#        print(x)
        return np.NaN

dataT = data.copy()
dataT['heard_pitch'] = dataT['heard_pitch'].apply(lambda x : pitch_to_int(x))
dataT['answered_pitch'] = dataT['answered_pitch'].apply(lambda x : pitch_to_int(x))

# compute the diff and abs diff

dataT['diff'] = dataT['heard_pitch'] - dataT['answered_pitch']
dataT['abs_diff'] = abs(dataT['heard_pitch'] - dataT['answered_pitch'])
dataT.head()

print(dataT.shape)
dataT = dataT.dropna()
print(dataT.shape)

(1256, 7)
(1254, 7)


#### T test :
A One Sample T-Test is a statistical test used to evaluate the null hypothesis. To begin, we assume that our null hypothesis is true. Then we compute the p-value, and if it is smaller or equal to $\alpha$, the null hypthesis is rejected.

$\alpha$ is the signifiance level (0.01, 0.05 or 0.10), which is the probability of makingna Type I error.* 

#### Null hypothesis : 
$H_{0}$ : The musicians answer by chance, with a mean error bigger than 3 ($\mu = 3$).

#### Alternative hypothesis : 
$H_{a}$ : The musicians answer mostly accurately, with a mean error lower than 3.

#### T value : 

$t = \frac{\bar{x} - \mu}{\sigma / \sqrt{n}}$

where $\bar{x}$ is the sample mean, $\sigma$ the population standard deviation and $n$ the sample size.

*A type I error (or error of the first kind) is the incorrect rejection of a true null hypothesis. A type II error (or error of the second kind) is the failure to reject a false null hypothesis.


In [3]:
#dataT.groupby('ID')['abs_diff'].mean().tolist()

In [4]:

true_mu = 3

onesample_results = scipy.stats.ttest_1samp(dataT.groupby('ID')['abs_diff'].mean().tolist(), true_mu)

matrix_onesample = [
    ['', 'Test Statistic', 'p-value'],
    ['Sample Data', onesample_results[0], onesample_results[1]]
]

onesample_table = FF.create_table(matrix_onesample, index=True)
py.iplot(onesample_table, filename='onesample-table')

/Users/Albane/anaconda2/lib/python2.7/site-packages/plotly/tools.py:1491: UserWarning:

plotly.tools.FigureFactory.create_table is deprecated. Use plotly.figure_factory.create_table



In [5]:
onesample_results

Ttest_1sampResult(statistic=-13.285338673936817, pvalue=3.1048411855410059e-15)

In [7]:
#sample = dataT.abs_diff.values

sample = dataT[dataT.timbre == 'piano'].abs_diff.values
x_bar = sample.mean()
sigma = sample.std()
n = sample.size
t = ((x_bar - true_mu) / (sigma / np.sqrt(n)))
t

-21.093776443115154

### Two sample T-test

sample 1 : string instr
sample 2 : others

#### Null hypothesis : 
The two group make equal performances 



The t statistic to test whether the means are different can be calculated as follows:

$ t={\frac {{\bar {X}}_{1}-{\bar {X}}_{2}}{s_{p}{\sqrt {\frac {2}{n}}}}}$
where
$s_{p}={\sqrt {\frac {s_{X_{1}}^{2}+s_{X_{2}}^{2}}{2}}}$

In [9]:
dataT.columns

Index([            u'ID',     u'Instrument',         u'timbre',
          u'heard_pitch', u'answered_pitch',           u'diff',
             u'abs_diff'],
      dtype='object')

In [24]:
instr_dict = { 'piano' : 'percussion', 'batterie' : 'percussion',  
              'violon' : 'string', 'alto' : 'string', 'violoncelle' : 'string', 'guitare' : 'string',
              'contrebasse':'string', 'basse':'string',
              'trompette' : 'wind', 'trombone' : 'wind', 'saxophone' : 'wind', 'hautbois' : 'wind', 
              'clarinette' : 'wind', 'cor' : 'wind', 'flute':'wind', 'chant':'wind'
} # j'ai mis 'chant' dans 'wind'...

def instr_to_class(x):
    if x in instr_dict.keys():
        return instr_dict[x]
    else:
        #print(x)
        return np.NaN

dataT['instr_class'] = dataT['Instrument'].apply(lambda x : instr_to_class(x))
a = dataT[dataT.instr_class == 'string'].groupby('ID')['abs_diff'].mean().tolist()
b = dataT[dataT.instr_class != 'string'].groupby('ID')['abs_diff'].mean().tolist()
b = b[2:18]
print(len(a), len(b))

(16, 16)


In [25]:
scipy.stats.ttest_rel(a, b)

Ttest_relResult(statistic=-2.4135392364772894, pvalue=0.029045491989680212)

### Two sample T-test

sample 1 : not quantified 
sample 2 : quantified

#### Null Hypothesis :
$H_{0}$ = $\mu_{1} = \mu_{2}$

#### Alternative Hypothesis :
$H_{a}$ = $\mu_{1} ≠ \mu_{2}$



In [57]:
quant_dict = { 'piano' : 1, 'batterie' : 0,  
              'violon' : 0, 'alto' : 0, 'violoncelle' : 0, 'guitare' : 1,
              'contrebasse':0, 'basse':1,
              'trompette' : 1, 'trombone' : 0, 'saxophone' : 1, 'hautbois' : 1, 
              'clarinette' : 1, 'cor' : 1, 'flute':1, 'chant':0
}

def instr_to_quant(x):
    if x in quant_dict.keys():
        return quant_dict[x]
    else:
        #print(x)
        return np.NaN

dataT['instr_quant'] = dataT['Instrument'].apply(lambda x : instr_to_quant(x))

a = dataT[dataT.instr_quant == 0].groupby('ID')['abs_diff'].mean().tolist() # not quantified
b = dataT[dataT.instr_quant == 1].groupby('ID')['abs_diff'].mean().tolist() # quantified 
b = b[:15]
print(len(a), len(b))

(15, 15)


In [58]:
scipy.stats.ttest_rel(a, b)

Ttest_relResult(statistic=-2.1436878051609054, pvalue=0.050102364509487213)

In [54]:
zero_sample = dataT[dataT.abs_diff == 0]
print(zero_sample[zero_sample.instr_quant == 0].shape,
zero_sample[zero_sample.instr_quant == 1].shape)

((244, 9), (174, 9))


In [61]:
#fretless vs fretted string
a = dataT[(dataT.instr_quant == 0) & (dataT.instr_class == 'string')].groupby('ID')['abs_diff'].mean().tolist() # not quantified
b = dataT[(dataT.instr_quant == 1) & (dataT.instr_class == 'string')].groupby('ID')['abs_diff'].mean().tolist() # quantified 
a = a[:6]
b = b[:15]
print(len(a), len(b))
scipy.stats.ttest_rel(a, b)

# t* = 4,032

(6, 6)


Ttest_relResult(statistic=-1.0252091820073903, pvalue=0.35228077479440489)

t\* = 4.032
So we have $-t* < t < t*$ so we have to accept H0

In [65]:
# tuned vs untuned percussions
a = dataT[(dataT.instr_quant == 0) & (dataT.instr_class == 'percussion')].groupby('ID')['abs_diff'].mean().tolist() # not quantified
b = dataT[(dataT.instr_quant == 1) & (dataT.instr_class == 'percussion')].groupby('ID')['abs_diff'].mean().tolist() # quantified 
b = b[:2]
print(len(a), len(b))
scipy.stats.ttest_rel(a, b)

#t* = 9,925

(2, 2)


Ttest_relResult(statistic=0.034482758620690002, pvalue=0.97805628876797779)

t\* = 9.925
So we have $-t* < t < t*$ so we have to accept H0